In [51]:
import pandas as pd
import matplotlib.pyplot as plt

In [52]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_countries_by_past_and_projected_GDP_(nominal)")

In [53]:
positions = 20

In [54]:
stuff = []
for example in tables:
    try:
        country_col = [x for x in example.columns if "Country" in str(x).split(" ")][0]
        years = [x for x in example.columns if "Country" not in str(x).split(" ")]
        example = example.rename(columns={country_col:"country"}).melt(id_vars="country",value_vars=years)
        example["country"] = [x.replace("*","").strip() for x in example["country"]]
        stuff.append(example)
    except Exception as e:
        print(str(e))

list index out of range
list index out of range
list index out of range
list index out of range


In [55]:
final = pd.concat(stuff).dropna()
final.rename(columns={"variable":"year","value":"gdp"},inplace=True)

In [56]:
final = final[final["year"].isin([str(x) for x in range(1970,2038)])]

In [57]:
allocation = final.sort_values(["year","gdp"],ascending=False).reset_index().groupby("year",sort=False).nth([i for i in range(20)]).reset_index()

In [58]:
interest_rates = pd.read_csv("./csv_files/API_FR.INR.RINR_DS2_en_csv_v2_5728810.csv",engine="c",encoding="ISO-8859-1",low_memory=False)

In [59]:
country_col = list(interest_rates.columns)[0]
interest_rates = interest_rates.rename(columns={country_col:"country"})
years = [x for x in interest_rates.columns[1:] if "Country" not in str(x).split(" ") and "Indicator" not in str(x).split(" ")]
interest_rates = interest_rates.melt(id_vars="country",value_vars=years).rename(columns={"variable":"year","value":"interest_rates"})
interest_rates = interest_rates[interest_rates["year"].isin([str(x) for x in range(1970,2038)])]

In [60]:
ir_cleaned = []
for country in interest_rates["country"].unique():
    sovereign = interest_rates[interest_rates["country"]==country].sort_values("year")
    sovereign.fillna(method="bfill",inplace=True)
    sovereign["interest_rates"] = (sovereign["interest_rates"] / 100) + 1
    ir_cleaned.append(sovereign)

In [61]:
completed_rates = pd.concat(ir_cleaned).dropna()

In [62]:
completed_rates

,country,year,interest_rates
2660,Aruba,1970,1.064754
2926,Aruba,1971,1.064754
3192,Aruba,1972,1.064754
3458,Aruba,1973,1.064754
3724,Aruba,1974,1.064754
...,...,...,...
15693,Zimbabwe,2018,0.356192
15959,Zimbabwe,2019,0.357048
16225,Zimbabwe,2020,0.188679
16491,Zimbabwe,2021,0.682044


In [63]:
start_year = 2000
end_year = 2020
complete = final.merge(completed_rates,on=["country","year"],how="left").dropna()
complete["year"] = [int(x) for x in complete["year"]]
complete = complete[(complete["year"]>=start_year) & (complete["year"]<=end_year)]
allocation = complete.sort_values(["year","gdp"],ascending=False).reset_index().groupby("year",sort=False).nth([i for i in range(20)]).reset_index()
allocation["position"] = [i % positions for i in allocation.index.values]


In [64]:
portfolio = allocation.pivot_table(index="year",columns="position",values="interest_rates").reset_index()
portfolio[[i for i in range(positions)]] = portfolio[[i for i in range(positions)]].cumprod()
portfolio["pv"] = [sum([row[1][x]*float(1/positions) for x in range(positions)]) for row in portfolio.iterrows()]
portfolio

position,year,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,pv
0,2000,1.068134,1.034495,1.045917,1.037107,1.051382,1.028135,1.051966,1.485047,1.083426,...,0.914340,1.018655,1.042034,0.983458,1.017520,1.048706,1.025679,1.064169,0.989143,1.052642
1,2001,1.116904,1.066438,1.079886,1.075756,1.094793,1.093614,1.094950,2.162790,1.176508,...,0.836017,1.037658,1.075044,1.020042,1.035348,1.106787,1.052017,1.120065,1.076667,1.120752
2,2002,1.151177,1.101650,1.098659,1.126099,1.129375,1.121679,1.127065,2.333806,1.745237,...,0.851613,1.081034,1.207515,1.037914,1.045183,1.138376,1.079031,1.024120,1.112157,1.182200
3,2003,1.175436,1.140052,1.111273,1.155807,1.158746,1.137244,1.159380,2.504358,1.759410,...,0.867500,1.116296,1.338556,1.056098,1.129297,1.149190,0.986601,1.050418,1.144477,1.239344
4,2004,1.194388,1.173390,1.129964,1.140980,1.190753,1.145324,1.153760,2.627325,2.529665,...,0.883684,1.159124,1.407283,1.074601,1.185202,1.160107,0.902088,1.077391,1.144872,1.299976
5,2005,1.229748,1.207484,1.147930,1.159387,1.229313,1.159693,1.668742,2.722147,2.652484,...,0.900169,1.191884,1.403824,1.093429,1.241635,1.060732,0.898893,1.060221,1.114267,1.346096
6,2006,1.287867,1.238261,1.172158,1.177780,1.271407,1.196310,2.356937,2.750871,2.720669,...,0.916962,1.214380,1.427102,1.112586,1.301361,0.969869,0.848304,1.072019,1.141110,1.400261
7,2007,1.354923,1.270541,1.169115,1.214364,1.319243,1.229042,3.182396,2.907172,2.766235,...,0.934068,1.242793,1.452105,1.163165,1.189886,1.003688,0.926239,1.175752,1.157733,1.478373
8,2008,1.397060,1.306713,1.142160,1.230224,1.376400,1.663714,3.205103,3.016823,2.832237,...,0.951493,1.194917,1.477547,1.063528,1.269429,1.071388,0.932301,1.279466,1.112542,1.526441
9,2009,1.433278,1.336846,1.205334,1.214960,1.418064,2.242554,3.359634,3.161890,2.859792,...,1.006185,1.222195,1.503434,0.972426,1.305005,1.266167,0.974929,1.389430,1.243626,1.609812


In [65]:
window = portfolio[(portfolio["year"]==start_year) & (portfolio["year"]==end_year)]
# plt.plot(portfolio["year"],portfolio["pv"])

In [67]:
allocation.head(20)

,year,index,country,gdp,interest_rates,position
0,2020,9135,United States,21060450.0,1.022104,0
1,2020,8985,China,14862564.0,1.038386,1
2,2020,9026,India,2671596.0,1.042039,2
3,2020,9032,Italy,1895692.0,1.007454,3
4,2020,8973,Brazil,1476092.0,1.211972,4
5,2020,9061,Mexico,1090515.0,1.020874,5
6,2020,9027,Indonesia,1062532.0,1.099859,6
7,2020,9117,Switzerland,739021.0,1.034012,7
8,2020,9122,Thailand,500525.0,1.046338,8
9,2020,9076,Nigeria,429423.0,1.053713,9
